## Ejecicio 4 DS
Definición de clases:

In [4]:
from abc import ABC, abstractmethod
import math
from enum import Enum

class EstadoMotor(Enum):
    ACELERANDO = 1
    FRENANDO = 2
    APAGADO = 3
    ENCENDIDO = 4

class Cliente:
    def __init__(self):
        self.gestor_filtros = None

    def setGestorFiltros(self, gestor_filtros):
        self.gestor_filtros = gestor_filtros

    def hacerPeticion(self):
        if self.gestor_filtros:
            self.gestor_filtros.peticionFiltros()
        else:
            print("Error: Gestor de filtros no configurado.")    
    
class Filtro(ABC):
    @abstractmethod
    def ejecutar(self, revoluciones, estado_motor):
        pass

class CadenaFiltros:
    def __init__(self):
        self.filtros = []
        self.objetivo = None

    def addFiltro(self, filtro):
        self.filtros.append(filtro)

    def ejecutarCadena(self):
        revoluciones = self.objetivo.revoluciones
        estado_motor = self.objetivo.estado_motor
        
        for filtro in self.filtros:
            revoluciones = filtro.ejecutar(revoluciones, estado_motor)
            
        self.objetivo.ejecutar(revoluciones, estado_motor)
        return revoluciones

    def setTarget(self, objetivo):
        self.objetivo = objetivo
        
class GestorFiltros:
    def __init__(self, objetivo):
        self.cadenaFiltros = CadenaFiltros()
        self.cadenaFiltros.setTarget(objetivo) #esto se puede gestionar de otra forma pero lo veo cómodo

    def addFiltro(self, filtro):
        self.cadenaFiltros.addFiltro(filtro)

    def peticionFiltros(self):
        self.cadenaFiltros.ejecutarCadena()
        

class RepercutirRozamiento(Filtro):
    def __init__(self):
        self.ROZAMIENTO = 1
    
    def ejecutar(self, revoluciones, estado_motor):
        
        revoluciones -= self.ROZAMIENTO  
        
        revoluciones = max(revoluciones, 0)
        
        return revoluciones


class CalcularVelocidad(Filtro):
    def __init__(self):
        self.VMAX = 5000
        self.incremento_velocidad = 0

    def ejecutar(self, revoluciones, estado_motor):
        if estado_motor == EstadoMotor.APAGADO or estado_motor == EstadoMotor.ENCENDIDO:
            incremento_velocidad = 0
        elif estado_motor == EstadoMotor.FRENANDO:
            incremento_velocidad = -100
        elif estado_motor == EstadoMotor.ACELERANDO:
            incremento_velocidad = 100

        # Aplicar incremento de velocidad
        revoluciones = revoluciones + incremento_velocidad
        
        # Aplicar límite máximo en la velocidad (5000 RPM)
        revoluciones = min(revoluciones, self.VMAX)
        
        return revoluciones

    
    
class Objetivo:
    def __init__(self):
        self.RADIO_EJE = 0.15

        self.revoluciones = 0
        self.velocidad = 0
        self.distancia = 0
        self.estado_motor = EstadoMotor.APAGADO

    def ejecutar(self, revoluciones, estado_motor):
        self.revoluciones = revoluciones
        self.estado_motor = estado_motor
        
        # Calcular la velocidad lineal en km/h
        self.velocidad = 2 * math.pi * self.RADIO_EJE * self.revoluciones * (60 / 1000)
        
        self.distancia += self.velocidad*(1./60.)
 
        
        # Devolvemos las revoluciones actualizadas
        return self.revoluciones


In [5]:
import tkinter as tk

class DispositivoControl(tk.Frame):
    def __init__(self, master=None, target=None):
        super().__init__(master)
        self.master = master
        self.master.title("Control del Vehículo")
        self.pack()

        self.target = target
        self.panel_mandos()
        self.salpicadero()

    def panel_mandos(self):
        self.panel_mandos = tk.LabelFrame(self, text="Panel de Mandos")
        self.panel_mandos.pack()

        self.estado_label = tk.Label(self.panel_mandos, text="Estado del Motor:")
        self.estado_label.pack()

        self.estado_motor_label = tk.Label(self.panel_mandos, text="", font=('Arial', 14, 'bold'))
        self.estado_motor_label.pack()

        self.encender_button = tk.Button(self.panel_mandos, text="Encender", command=self.toggle_encender)
        self.encender_button.pack(side="left")

        self.acelerar_button = tk.Button(self.panel_mandos, text="Acelerar", command=self.toggle_acelerar)
        self.acelerar_button.pack(side="left")

        self.frenar_button = tk.Button(self.panel_mandos, text="Frenar", command=self.toggle_frenar)
        self.frenar_button.pack(side="left")

    def salpicadero(self):
        self.salpicadero_frame = tk.LabelFrame(self, text="Salpicadero")
        self.salpicadero_frame.pack()

        self.velocimetro_label = tk.Label(self.salpicadero_frame, text="Velocímetro:")
        self.velocimetro_label.pack()

        self.velocimetro_value = tk.Label(self.salpicadero_frame, text="")
        self.velocimetro_value.pack()

        self.cuenta_revoluciones_label = tk.Label(self.salpicadero_frame, text="Cuentarrevoluciones:")
        self.cuenta_revoluciones_label.pack()

        self.cuenta_revoluciones_value = tk.Label(self.salpicadero_frame, text="")
        self.cuenta_revoluciones_value.pack()

        self.cuenta_kilometros_label = tk.Label(self.salpicadero_frame, text="Cuentakilómetros:")
        self.cuenta_kilometros_label.pack()

        self.cuenta_kilometros_value = tk.Label(self.salpicadero_frame, text="")
        self.cuenta_kilometros_value.pack()

        self.update_salpicadero()

    def update_salpicadero(self):
        if self.target:
            self.estado_motor_label.config(text=self.target.estado_motor.name)
            self.velocimetro_value.config(text=str(self.target.velocidad))
            self.cuenta_revoluciones_value.config(text=str(self.target.revoluciones))
            self.cuenta_kilometros_value.config(text=str(self.target.distancia))

        self.after(1000, self.update_salpicadero)

    def toggle_encender(self):
        if self.target:
            if self.target.estado_motor == EstadoMotor.APAGADO:
                self.target.estado_motor = EstadoMotor.ENCENDIDO
                self.encender_button.config(text="Apagar")
            else:
                self.target.estado_motor = EstadoMotor.APAGADO
                self.encender_button.config(text="Encender")

        self.update_estado_motor_label()

    def toggle_acelerar(self):
        if self.target:
            if self.target.estado_motor == EstadoMotor.ENCENDIDO or self.target.estado_motor == EstadoMotor.FRENANDO:
                self.target.estado_motor = EstadoMotor.ACELERANDO
                
            elif self.target.estado_motor == EstadoMotor.ACELERANDO:
                self.target.estado_motor = EstadoMotor.ENCENDIDO

        self.update_estado_motor_label()

    def toggle_frenar(self):
        if self.target:
            if self.target.estado_motor == EstadoMotor.ENCENDIDO or self.target.estado_motor == EstadoMotor.ACELERANDO:
                self.target.estado_motor = EstadoMotor.FRENANDO
                
            elif self.target.estado_motor == EstadoMotor.FRENANDO:
                self.target.estado_motor = EstadoMotor.ENCENDIDO

        self.update_estado_motor_label()

    def update_estado_motor_label(self):
        if self.target:
            estado_motor = self.target.estado_motor.name
            if estado_motor == "ENCENDIDO":
                self.estado_motor_label.config(text=estado_motor, fg="black", bg="yellow", font=('Arial', 16, 'bold'))
            elif estado_motor == "APAGADO":
                self.estado_motor_label.config(text=estado_motor, fg="black", bg="grey", font=('Arial', 12))
            elif estado_motor == "ACELERANDO":
                self.estado_motor_label.config(text=estado_motor, fg="black", bg="green", font=('Arial', 14, 'bold'))
            elif estado_motor == "FRENANDO":
                self.estado_motor_label.config(text=estado_motor, fg="white", bg="red", font=('Arial', 14, 'bold'))


In [6]:

def main():
    root = tk.Tk()

    # Crear el objeto Objetivo
    target = Objetivo()
        
    gestor = GestorFiltros(target)
    
    gestor.addFiltro(RepercutirRozamiento())
    gestor.addFiltro(CalcularVelocidad())
    

    
    cliente = Cliente()
    cliente.setGestorFiltros(gestor)
    
    app = DispositivoControl(master=root, target=target)
    
    # Agregar lógica para que en cada tick se hagan peticiones desde el cliente
    def tick():       
        # Realizar la petición al gestor de filtros
        cliente.hacerPeticion()
        
        # Actualizar el salpicadero de la interfaz gráfica
        app.update_salpicadero()
        
        # Llamar a la función tick nuevamente después de un tiempo
        root.after(1000, tick)  # Llama a la función tick cada 1000 milisegundos (1 segundo)
    
    # Llamar a la función tick inicialmente para iniciar el bucle
    tick()
    
    root.mainloop()

if __name__ == "__main__":
    main()